In [1]:
import h5py, os
import numpy as np

In [12]:
f = h5py.File(os.getcwd()+'/data/lhe_data.h5', 'r+')
#f = h5py.File(os.getcwd()+'/data/lhe_data_shuffled.h5', 'r+')

In [3]:
f.keys()

<KeysViewHDF5 ['benchmarks', 'morphing', 'observables', 'parameters', 'sample_summary', 'samples']>

In [4]:
bm = f['benchmarks']
bm.keys()

<KeysViewHDF5 ['is_nuisance', 'is_reference', 'names', 'values']>

In [5]:
mp = f['morphing']
mp.keys()

<KeysViewHDF5 ['components', 'morphing_matrix']>

In [6]:
ob = f['observables']
ob.keys()

<KeysViewHDF5 ['definitions', 'names']>

In [7]:
pm = f['parameters']
pm.keys()

<KeysViewHDF5 ['lha_blocks', 'lha_ids', 'max_power', 'names', 'ranges', 'transforms']>

In [8]:
ss = f['sample_summary']
ss.keys()

<KeysViewHDF5 ['background_events', 'signal_events_per_benchmark']>

In [13]:
samples = f['samples']
samples.keys()

<KeysViewHDF5 ['observations', 'sampling_benchmarks', 'weights']>

In [10]:
np.array(f['samples/sampling_benchmarks'])

array([7, 6, 4, ..., 4, 8, 4])

In [11]:
f.close()

### Samples

In [14]:
observations = np.array(samples['observations'])
observations.shape

(109979, 3)

In [15]:
weights = np.array(samples['weights'])
weights

array([[6.4833075e-08, 1.5697967e-07, 1.0588605e-07, ..., 1.0607000e-07,
        1.1375792e-07, 1.2988714e-07],
       [8.8350620e-08, 9.2981912e-08, 9.0654617e-08, ..., 9.0663855e-08,
        9.1043360e-08, 9.1801846e-08],
       [9.0181785e-08, 1.1667266e-07, 1.0289295e-07, ..., 1.0294580e-07,
        1.0512995e-07, 1.0956984e-07],
       ...,
       [8.8997676e-08, 1.1804735e-07, 1.0302750e-07, ..., 1.0308546e-07,
        1.0547797e-07, 1.1032702e-07],
       [1.0989271e-07, 1.1590488e-07, 1.1287510e-07, ..., 1.1288710e-07,
        1.1338000e-07, 1.1436650e-07],
       [1.0544206e-07, 9.7514887e-08, 1.0109032e-07, ..., 1.0107452e-07,
        1.0043663e-07, 9.9226820e-08]])

In [16]:
sampling_benchmarks = np.array(samples['sampling_benchmarks'])
print('num of sm events: '+str(sum(sampling_benchmarks == 3)))
print('num of 50 events: '+str(sum(sampling_benchmarks == 4)))
print('num of neg_50 events: '+str(sum(sampling_benchmarks == 5)))
print('num of 200 events: '+str(sum(sampling_benchmarks == 6)))
print('num of neg_200 events: '+str(sum(sampling_benchmarks == 7)))
print('num of 500 events: '+str(sum(sampling_benchmarks == 8)))
print('num of neg_500 events: '+str(sum(sampling_benchmarks == 9)))

num of sm events: 49993
num of 50 events: 9999
num of neg_50 events: 9999
num of 200 events: 9997
num of neg_200 events: 9997
num of 500 events: 9997
num of neg_500 events: 9997


### Embedding toy data

In [18]:
toydataFile = h5py.File(os.getcwd()+'/toydata/gphi_toydata.h5', 'r')

#### data

In [19]:
del f['samples/observations']
f.create_dataset('samples/observations', data=np.array(toydataFile['Data']))

<HDF5 dataset "observations": shape (6000000, 9), type "<f4">

#### sampling_benchmarks

In [20]:
N = 500000
sampling_benchmarks = np.concatenate([np.concatenate([np.ones(N)*3, np.ones(N)*i]) for i in range(4, 10)])
del f['samples/sampling_benchmarks']
f.create_dataset('samples/sampling_benchmarks', data=sampling_benchmarks)

<HDF5 dataset "sampling_benchmarks": shape (6000000,), type "<f8">

#### weights

In [21]:
Weights = np.array(toydataFile['Weights'])

In [22]:
Weights

array([7.1692028e-08, 7.1692028e-08, 7.1692028e-08, ..., 4.0921014e-07,
       4.0921014e-07, 4.0921014e-07], dtype=float32)

In [23]:
weights = np.ones([N*12, 10])*(1e10)

In [25]:
#weights[N*0:N*1, 3] = Weights[N*0]
#weights[N*1:N*2, 4] = Weights[N*1]
#weights[N*:N*1, 3] = Weights[N*0]

In [24]:
for i in range(12):
    if i%2==0:
        print('sm data, pos=%d'%(3))
        weights[N*i:N*(i+1), 3] = Weights[N*i]
    else:
        print('bsm data, pos=%d'%(3+(i+1)/2))
        weights[N*i:N*(i+1), int(3+(i+1)/2)] = Weights[N*i]

sm data, pos=3
bsm data, pos=4
sm data, pos=3
bsm data, pos=5
sm data, pos=3
bsm data, pos=6
sm data, pos=3
bsm data, pos=7
sm data, pos=3
bsm data, pos=8
sm data, pos=3
bsm data, pos=9


In [25]:
del f['samples/weights']
f.create_dataset('samples/weights', data=weights)

<HDF5 dataset "weights": shape (6000000, 10), type "<f8">

#### observable numbers

In [26]:
f['observables'].keys()

<KeysViewHDF5 ['definitions', 'names']>

In [27]:
print(np.array(f['observables/definitions']))
print(np.array(f['observables/names']))

[b'j[0].pt' b'j[0].deltaphi(j[1]) * (-1. + 2.*float(j[0].eta > j[1].eta))'
 b'met.pt']
[b'pt_j1' b'delta_phi_jj' b'met']


In [28]:
del f['observables/definitions']
del f['observables/names']

observables = ['s', 'theta', 'thetaZ', 'thetaW', 'Sin(phiZ)', 'Sin(phiW)', 'Cos(phiZ)', 'Cos(phiW)', 'Pt']
f.create_dataset('observables/definitions', data=(np.array(observables, dtype='S')))
f.create_dataset('observables/names', data=(np.array(observables, dtype='S')))

<HDF5 dataset "names": shape (9,), type "|S9">

In [29]:
f.close()